In [1]:
import json
import torch
import os
import numpy as np
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from sentence_transformers import SentenceTransformer
from torch.optim import Adam
from torch.nn.functional import cosine_similarity

2025-03-09 15:34:09.585300: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 15:34:09.779596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741514649.846863 3358202 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741514649.857944 3358202 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 15:34:10.061411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
torch.set_default_dtype(torch.float16)

# Initialize Retriever Model for Multi-Hop Retrieval (FAISS/BM25 can be added)
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
# RL Parameters
LEARNING_RATE = 5e-5
GAMMA = 0.9  # Discount factor
ALPHA = 0.1  # Exploration-exploitation balance

In [26]:
# Define reward function based on similarity between model output and ground truth
def compute_reward(model_answer, gold_answer):
    model_embedding = retriever_model.encode(model_answer, convert_to_tensor=True)
    gold_embedding = retriever_model.encode(gold_answer, convert_to_tensor=True)
    return cosine_similarity(model_embedding, gold_embedding, dim=0).item()


In [27]:
def save_list_to_json(lst, filename):
    """Save list to JSON file."""
    with open(filename, 'w') as file:
        json.dump(lst, file)

def rm_file(file_path):
    """Delete file if exists."""
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File {file_path} removed successfully.")

In [28]:
# Load Model and Tokenizer
model_name = "microsoft/DialoGPT-large"
save_file = f'qa_output/gpt_rl.json'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

prefix = "Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly."


In [29]:
# Load Data
with open('step1_data.json', 'r') as file:
    doc_data = json.load(file)

# RL Optimizer
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

In [30]:
def retrieve_context(query, retrieval_list, hops=2):
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.unsqueeze(0)  # Ensure it is 2D

    scored_docs = sorted(
        retrieval_list,
        key=lambda doc: torch.nn.functional.cosine_similarity(
            query_embedding,
            retriever_model.encode(doc['text'], convert_to_tensor=True).unsqueeze(0),  # Ensure 2D shape
            dim=1
        ).item(),
        reverse=True
    )

    # Multi-hop refinement
    context = ""
    for i in range(min(hops, len(scored_docs))):
        context += scored_docs[i]['text'] + "\n\n"

    return context


In [31]:
# Query Model
def query_bot(messages, temperature=0.1, max_new_tokens=512, **kwargs):
    """Generate a response using the model."""
    max_length = 512
    inputs = tokenizer(messages, max_length=max_length, truncation=True, return_tensors='pt')
    input_ids, attention_mask = inputs.input_ids, inputs.attention_mask
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

    generation_config = GenerationConfig(temperature=temperature, do_sample=True, **kwargs)

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            max_new_tokens=max_new_tokens,
            return_dict_in_generate=True  # Ensure it returns a dictionary
        )

    # Corrected line: Extract sequences properly
    output = tokenizer.decode(generation_output["sequences"][0], skip_special_tokens=True)

    return output

In [32]:
# Training Loop with RL
rm_file(save_file)
save_list = []

for d in tqdm(doc_data):
    retrieval_list = d['retrieval_list']
    
    # Multi-hop retrieval
    context = retrieve_context(d['query'], retrieval_list, hops=2)
    prompt = f"{prefix}\n\nQuestion: {d['query']}\n\nContext:\n\n{context}"
    
    # Generate answer
    response = query_bot(prompt)
    print(response)

    # Compute reward based on similarity with the ground truth
    reward = compute_reward(response, d['answer'])
    
    # RL Optimization: Update model weights based on reward
    loss = -reward  # Negative loss to maximize reward
    optimizer.zero_grad()
    torch.tensor(loss, requires_grad=True).backward()
    optimizer.step()
    
    # Store results
    save = {
        'query': d['query'],
        'prompt': prompt,
        'model_answer': response,
        'gold_answer': d['answer'],
        'question_type': d['question_type'],
        'reward': reward
    }
    save_list.append(save)

save_list_to_json(save_list, save_file)

  0%|          | 0/1194 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 2/1194 [00:00<01:04, 18.39it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  0%|          | 4/1194 [00:00<01:07, 17.67it/s]

Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly.

Question: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?

Context:

[Excerpt from document]
title: The FTX trial is bigger than Sam Bankman-Fried
published_at: 2023-09-28T12:00:00+00:00
source: The Verge
Excerpt:
-----
Some additional trouble for the crypto industry is likely to come from one crucial element of the fraud trial — the part where the government must prove intent.

The first part of proving the government’s case is pretty simple and a little boring: prosecutors must show that certain transactions took place. Whatever records the Southern District of New York has for the transactions will be shown.

“What conversations ha

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  1%|          | 6/1194 [00:00<01:08, 17.44it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  1%|          | 8/1194 [00:00<01:12, 16.33it/s]

Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly.

Question: Which online betting platform provides a welcome bonus of up to $1000 in bonus bets for new customers' first losses, runs NBA betting promotions, and is anticipated to extend the same sign-up offer to new users in Vermont, as reported by both CBSSports.com and Sporting News?

Context:

[Excerpt from document]
title: Vermont Sportsbook Promos and Sports Betting Launch News
published_at: 2023-09-26T22:53:28+00:00
source: Sporting News
Excerpt:
-----
2023. The site has already begun taking advantage of various tie-ins with the renowned sports programming network which has helped in the effort to establish the site in the overall marketplace.

In other states, the ESPN Bet welcome offer has included a 100% deposit match bonus worth up to $1,000 (with a 20x wagering requirement). ESPN Bet has also run a welcome prom

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  1%|          | 10/1194 [00:00<01:17, 15.35it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly.

Question: Between the TechCrunch report on the SBF trial published on October 6, 2023, and the TechCrunch report on Sam Bankman-Fried's prosecution allegations published on October 7, 2023, was there consistency in the portrayal of Sam Bankman-Fried's actions related to the FTX collapse?

Context:

[Excerpt from document]
title: SBF Trial: The latest updates from the FTX collapse’s courtroom drama
published_at: 2023-10-06T17:22:02+00:00
source: TechCrunch
Excerpt:
-----
SBF trial: Everything to know from the FTX courtroom ahead of his testimony Sam Bankman-Fried is in court for alleged fraud and money laundering while leading crypto exchange FTX and sister company Alameda. Here’s our latest coverage as the case nears its end.

The trial of Bankman-Fried, the former CEO and co-founder of collapsed crypto exchange FTX, is 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 99%|█████████▉| 1188/1194 [01:15<00:00, 18.06it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|█████████▉| 1190/1194 [01:15<00:00, 18.16it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly.

Question: Who is the individual that, according to articles from The Verge, Fortune, and TechCrunch, used FTX customer funds to secure a buyout, recruited a colleague from Jane Street for ventures at Alameda and FTX, and is facing allegations of committing fraud for personal gain?

Context:

[Excerpt from document]
title: The FTX trial is bigger than Sam Bankman-Fried
published_at: 2023-09-28T12:00:00+00:00
source: The Verge
Excerpt:
-----
According to a now-deleted profile from FTX investors Sequoia Capital, FTX was founded because of Bankman-Fried’s frustration with other exchanges when he was running Alameda Research, his crypto trading firm. According to the SEC, FTX was a fraud “from the start,” diverting customers’ funds to Alameda.

Sam Bankman-Fried, the founder of failed cryptocurrency exchange FTX, is on trial 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|█████████▉| 1192/1194 [01:15<00:00, 17.47it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 1194/1194 [01:15<00:00, 15.71it/s]

Below is a question followed by context from different sources. Please answer using the context. If insufficient, respond 'Insufficient Information'. Answer directly.

Question: Who is the individual mentioned in articles from both The Verge and TechCrunch, who cited challenges in managing a rapidly growing FTX and Alameda Research, and is also accused by the prosecution of committing fraud for personal gain?

Context:

[Excerpt from document]
title: The FTX trial is bigger than Sam Bankman-Fried
published_at: 2023-09-28T12:00:00+00:00
source: The Verge
Excerpt:
-----
If he wrote things on Twitter or said things in interviews that weren’t true, that’s more fodder for the government’s case. “They can point to that and say, ‘This is what he said, this is what actually happened.’”

Other evidence may include Signal messages and testimony from co-conspirators who plead guilty to their own charges. Alameda Research CEO Caroline Ellison, who was also sometimes Bankman-Fried’s girlfriend, may

In [ ]:
from evaluate import load

test_data = load_dataset("trivia_qa", "rc", split="test[:100]")

doc_data = []
for example in test_data:
    query = example["question"]
    # Use available evidence if present; otherwise, provide a dummy context.
    if "evidence" in example and len(example["evidence"]) > 0:
        retrieval_list = [{"text": example["evidence"][0]["text"]}]
    else:
        retrieval_list = [{"text": "No additional context available."}]
    # Assume the answer is stored as a string or in a dict.
    answer = example["answer"]["value"] if isinstance(example["answer"], dict) else example["answer"]
    doc_data.append({
        "query": query,
        "retrieval_list": retrieval_list,
        "answer": answer,
        "question_type": "trivia"
    })

# Load evaluation metrics
bleu = load("bleu")
rouge = load("rouge")
bert_score = load("bertscore")

# Generate predictions
predictions = []
ground_truths = []

D = []

for data in tqdm(doc_data):
    retrieval_list = data['retrieval_list']
    context = retrieve_context(data['query'], retrieval_list, hops=2)
    prompt = f"{prefix}\n\nQuestion: {data['query']}\n\nContext:\n\n{context}"
    gold_answer = data['answer']
    model_answer, _ = query_bot_rl(prompt)

    predictions.append(model_answer)
    ground_truths.append(gold_answer)
    
    reward = compute_reward(model_answer, gold_answer)
    D.append(reward)



In [ ]:
# Compute Exact Match (EM)
exact_match = sum([1 if pred.lower().strip() == gold.lower().strip() else 0 
                   for pred, gold in zip(predictions, ground_truths)]) / len(ground_truths)

# Compute BLEU Score
bleu_score = bleu.compute(predictions=predictions, references=ground_truths)

# Compute ROUGE Score
rouge_score = rouge.compute(predictions=predictions, references=ground_truths)

# Compute BERTScore
bert_score_result = bert_score.compute(predictions=predictions, references=ground_truths, lang="en")
avg_bert_score = sum(bert_score_result['f1']) / len(bert_score_result['f1'])

# Compute average reward
avg_reward = np.mean(D)

# Print Results
print(f"Exact Match (EM): {exact_match:.4f}")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print(f"ROUGE-L Score: {rouge_score['rougeL']:.4f}")
print(f"BERTScore (F1): {sum(bert_score_result['f1']) / len(bert_score_result['f1']):.4f}")
print(f"Average Reward: {avg_reward:.4f}")

# # Save results to a file
# results = {
#     'exact_match': exact_match,
#     'bleu': bleu_score['bleu'],
#     'rougeL': rouge_score['rougeL'],
#     'bert_score': avg_bert_score,
#     'average_reward': avg_reward
# }

# print(results)
print("Evaluation completed.")